In [ ]:
import warnings

warnings.filterwarnings('ignore')

import os
import cv2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.optimizers import SGD, Adam

from keras.callbacks import EarlyStopping

In [ ]:
# Funciones útiles.

def leerImagenes(path, label, im = 500, shape = (256, 256, 1)): # Para leer.
    
    im = [cv2.imread(path + os.listdir(path)[i]) for i in range(0, im)] # Leemos.
    im = [cv2.cvtColor(cv2.resize(i, shape), cv2.COLOR_BGR2GRAY) for i in im] # Resize y blanco/negro.
    
    return [[i/np.max(i), label] for i in im] # Normalización y agregamos la label.

def leerImagenes_tensor(path, label, im = 500, shape = (256, 256), MAX_IM = 1800): # Para leer imágenes pero ocupando tensors.
    
    max_im = np.max([len(os.listdir(i)) for i in PATH_IM]) # Número máximo de imágenes.
    lista_im = os.listdir(path)
    im_faltantes = MAX_IM - len(lista_im) # Imágenes faltantes de cada clase.
    
    im = [cv2.imread(path + lista_im[i]) for i in range(0, len(lista_im))] # Leemos las imágenes.
    
    rsz_rescale = tf.keras.Sequential([tf.keras.layers.Resizing(256, 256),
                                       tf.keras.layers.Rescaling(1./255)]) # Resize y normalización de cada una de las imágenes.
    
    rsz_im = [tf.image.rgb_to_grayscale(rsz_rescale(i)).numpy() for i in im] # Reescalamiento y reescalamiento.
    
    nueva = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal"), # Modelo para generar nuevas imágenes.
                                 tf.keras.layers.RandomRotation(factor = (-0.10, 0.10)),
                                 tf.keras.layers.RandomContrast(factor = (0, 1))])
    
    extra_im = [nueva(rsz_im[np.random.randint(0, len(lista_im))]).numpy() for i in range(0, im_faltantes)] # Imágenes extra.
    
    total_im = np.concatenate((rsz_im, extra_im), axis = 0) # Dataset completo.
    
    return [[i, label] for i in total_im] # Imagen con su label.

def vgg16Model(input = (256, 256, 1), output = 6):
    
    kernel = (3, 3)
    
    modelo = tf.keras.Sequential() # Modelo secuencial.
    modelo.add(tf.keras.layers.Conv2D(64, kernel_size = kernel, padding = 'same', activation = 'relu', input_shape = input))
    modelo.add(tf.keras.layers.Conv2D(64, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2)) #
    
    modelo.add(tf.keras.layers.Conv2D(128, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(128, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))
    
    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))
    
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))
    
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.Conv2D(512, kernel_size = kernel, padding = 'same', activation = 'relu'))
    modelo.add(tf.keras.layers.MaxPooling2D(2, 2))
    
    modelo.add(tf.keras.layers.Flatten())
    
    modelo.add(tf.keras.layers.Dense(4096, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(4096, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(1000, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(256, activation = "relu"))
    modelo.add(tf.keras.layers.Dense(6, activation = tf.keras.activations.softmax))
    
    return modelo
    

In [ ]:
# Lectura de los datos.

np.random.seed(42)

PATH_GENERAL = '/kaggle/input/enfpulm/enfermedadesPulmonares/'
paths = sorted(os.listdir(PATH_GENERAL))

PATH_IM = [PATH_GENERAL + i + '/' for i in paths] # Paths de las imágenes.

labels = [[1, 0, 0, 0, 0, 0],
          [0, 1, 0, 0, 0, 0],
          [0, 0, 1, 0, 0, 0],
          [0, 0, 0, 1, 0, 0],
          [0, 0, 0, 0, 1, 0],
          [0, 0, 0, 0, 0, 1]] # Labels.

print([len(os.listdir(i)) for i in PATH_IM])

data = [leerImagenes_tensor(PATH_IM[i], labels[i], MAX_IM = 2000) for i in range(0, len(PATH_IM))] # Imágenes y labels.

print("Shape de los datos {}".format(np.shape(data)))

div = int(len(data[0]) * 0.8) # Número de imágenes (división 80 - 20)
test = len(data[0]) - div

print("División {}".format(div))

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
     # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
    
except ValueError: tpu = None

if tpu:
    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    st = tf.distribute.experimental.TPUStrategy(tpu)
    
else:
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
    st = tf.distribute.get_strategy()

print("REPLICAS: ", st.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH = 32 * st.num_replicas_in_sync
N_ITER = 250

n, lr, ep = 3, 0.1, 60


es = EarlyStopping(monitor="val_loss", patience = 15)

modelos, resultados = [], []

callbacks_list = [es]

for i in range(0, n): # Número de entrenamientos.
    
    print("Entrenamiento: {}".format(i + 1))
    
    aux_train, aux_test = [i[0:div] for i in data], [i[div:len(i)] for i in data] # Separación de train y test de cada una de las clases.
    train, test = aux_train[0], aux_test[0] # Primer set de imágenes.
    
    for i in range(1, len(data)): # Construimos el dataset.

        train, test = np.concatenate((train, aux_train[i]), axis = 0), np.concatenate((test, aux_test[i]), axis = 0) # Concatenamos todo el dataset.

    #train, test = shuffle(train), shuffle(test) # Shuffle de los datos de entrenamiento y prueba.

    img_train, img_test = np.array([train[i][0] for i in range(len(train))]), np.array([test[i][0] for i in range(len(test))]) # Imágenes de entrenamiento y testeo.
    lbl_train, lbl_test = np.array([train[i][1] for i in range(len(train))]), np.array([test[i][1] for i in range(len(test))]) # Etiquetas de entrenamiento y testeo.
    
    # Una vez seperados los datos convertimos a datasets de tensorflow.
    
    train_dataset = (tf.data.Dataset.from_tensor_slices((img_train.astype(np.float32),
                                                         lbl_train.astype(np.float32)))
                    .repeat()
                    .shuffle(2048)
                    .batch(BATCH)
                    .prefetch(AUTO))
    
    val_dataset = (tf.data.Dataset.from_tensor_slices((img_test.astype(np.float32),
                                                       lbl_test.astype(np.float32)))
                   .shuffle(2048)
                   .batch(BATCH)
                   .prefetch(AUTO))
    
    test_dataset = (tf.data.Dataset.from_tensor_slices(img_test.astype(np.float32))
                    .batch(BATCH))
    
    # Entrenamos el modelo con TPU
    
    with st.scope(): modelo = vgg16Model() # Creamos el modelo.
        
    modelo.compile(optimizer = SGD(learning_rate = lr, decay = lr/ep),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    historia = modelo.fit(train_dataset,
                          steps_per_epoch = N_ITER,
                          epochs = ep,
                          validation_data = (val_dataset),
                          callbacks = callbacks_list)
    
    resultados.append(historia)
    modelos.append(modelo)

In [ ]:
!mkdir '/kaggle/working/TPU_rad01/'
!mkdir '/kaggle/working/TPU_rad01/history/'
!mkdir '/kaggle/working/TPU_rad01/model'

In [ ]:
keys = resultados[0].history.keys()

PATH = '/kaggle/working/TPU_rad01/history/' # Path para guardar los resultados.

for i in range(0, len(resultados)):
    
    for j in keys: # Recorremos todas las keys de los modelos.
        
        PATH_AUX = PATH + 'modelo_{}_'.format(i + 1) + j + '.txt'
        
        with open(PATH_AUX, 'w') as f:
            
            for k in resultados[i].history[j]:
                
                f.write(str(k) + '\n')

In [ ]:
PATH_MODELOS = '/kaggle/working/TPU_rad01/model/'

for i in range(0, len(modelos)):
    
    modelos[i].save(PATH_MODELOS + 'modelo01_{}'.format(i + 1))
    conv = tf.lite.TFLiteConverter.from_saved_model(PATH_MODELOS + 'modelo01_{}'.format(i + 1))
    tflite = conv.convert()
    
    with open(PATH_MODELOS + 'modelo01_{}/'.format(i + 1) + 'modelo01_{}.tflite'.format(i + 1), 'wb') as f:
    
        f.write(tflite)

In [ ]:
print("hola")